## `SSIM (original_tomogram, denoise(original_tomogram + noise))`
The higher the index, the better the denoising.

In [ ]:
# Configuration
original_tomogram_name = "empiar10311_stack_crop.mrc"
mean_noise = 0.0
sigma_noise = 1.5
sigma_denoising = 2.5

In [ ]:
# Mount Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
#import scipy.ndimage
#import matplotlib.pyplot as plt
!pip install mrcfile
import mrcfile
import scipy.stats
from skimage.metrics import structural_similarity as SSIM
import time
import _3D_denoising
import kernels

In [ ]:
def add_gaussian_noise(tomogram, mean, sigma):
  noise = np.random.normal(mean,sigma,tomogram.shape)
  return tomogram + noise 

In [ ]:
def compute_SSIM(original_tomogram, denoised_tomogram, border_size):
  assert border_size > 0
  ROI_original_tomogram = original_tomogram[border_size:-border_size, border_size:-border_size, border_size:-border_size].flatten()
  ROI_denoised_tomogram = denoised_tomogram[border_size:-border_size, border_size:-border_size, border_size:-border_size].flatten()
  index = SSIM(ROI_original_tomogram, ROI_denoised_tomogram, data_range=ROI_original_tomogram.max() - ROI_denoised_tomogram.min(), full=False)
  return index

In [ ]:
original_tomogram_MRC = mrcfile.open(f'drive/Shareddrives/TomogramDenoising/tomograms/{original_tomogram_name}')
original_tomogram = original_tomogram_MRC.data[0:50, 0:50, 0:50]
print(original_tomogram.dtype, original_tomogram.shape)
#denoised_tomogram_MRC = mrcfile.open(f'drive/Shareddrives/TomogramDenoising/results/{denoised_tomogram_name}')
#denoised_tomogram = denoised_tomogram_MRC.data
#print(denoised_tomogram.dtype, denoised_tomogram.shape)

In [ ]:
noised_tomogram = add_gaussian_noise(original_tomogram, mean_noise, sigma_noise)

In [ ]:
kernel = kernels.get_gaussian_kernel(sigma_denoising)
denoised_tomogram = _3D_denoising.filter(noised_tomogram, kernel)

In [ ]:
border_size = int(sigma_denoising * 4)
index = compute_SSIM(original_tomogram, denoised_tomogram, border_size)

In [ ]:
output = f"tomogram=\"{original_tomogram_name}\"\t sigma_noise={sigma_noise}\t sigma_denoising={sigma_denoising}\t SSIM_index={index}"
print(output)

In [ ]:
with open("drive/Shareddrives/TomogramDenoising/results/SSIM_indexes_after_adding_noise.txt", 'a') as f:
  f.write(output + '\n')

In [ ]:
print(f"time: {time.process_time()} seconds")